In [1]:
using CSV, DataFrames, Statistics, Dates, Gadfly, Random;
include("utils/precipitation.jl");

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/chaime/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


In [10]:
function generate_variation(val)
    var = rand() * 0.4;
    return val * (0.8 + var);
end

generate_variation (generic function with 1 method)

In [11]:
function generate_new_entry(df, entry)
    nwDATE = entry.DATE;
    nwSURVERSE = entry.SURVERSE;
    nwFS_dist = entry.FS_dist;
    nwSS_dist = entry.SS_dist;
    nwFS_sum = generate_variation(entry.FS_sum);
    nwFS_max = generate_variation(entry.FS_max);
    nwFS_max3 = generate_variation(entry.FS_max3);
    nwSS_sum = generate_variation(entry.SS_sum);
    nwSS_max = generate_variation(entry.SS_max);
    nwSS_max3 = generate_variation(entry.SS_max3);
    
    push!(df, [nwDATE, nwSURVERSE, nwFS_dist, nwSS_dist, 
               nwFS_sum, nwFS_max, nwFS_max3, 
               nwSS_sum, nwSS_max, nwSS_max3])
end

generate_new_entry (generic function with 1 method)

## Ouvrage 3260

In [16]:
data_set = CSV.read("data/parsed/ouvrage_3260.csv");
size(data_set)

(1097, 10)

In [17]:
first(shuffleDf(data_set), 10)

,DATE,SURVERSE,FS_dist,SS_dist,FS_sum,FS_max,FS_max3,SS_sum
,Date,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,2016-08-30,0,0.145981,0.210769,-0.396293,-0.415792,-0.417149,-0.268446
2,2014-07-04,0,0.145981,0.210769,-0.396293,-0.415792,-0.417149,-0.113033
3,2016-05-13,0,0.145981,0.210769,0.0260856,0.445,0.253495,0.221703
4,2014-05-04,0,0.145981,0.210769,2.30437,0.731931,1.08672,-0.292356
5,2015-08-18,0,0.145981,0.210769,2.12518,5.86481,3.58639,-0.292356
6,2013-05-06,0,0.145981,0.210769,-0.396293,-0.415792,-0.417149,-0.292356
7,2013-10-12,0,0.145981,0.210769,-0.396293,-0.415792,-0.417149,-0.292356
8,2014-09-17,0,0.145981,0.210769,-0.345096,-0.35203,-0.335858,-0.292356
9,2014-10-27,0,0.145981,0.210769,-0.396293,-0.415792,-0.417149,-0.292356


In [18]:
describe(data_set)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1097,,Date
2,SURVERSE,0.0856882,0,0.0,1,,,Int64
3,FS_dist,0.145981,0.145981,0.145981,0.145981,,,Float64
4,SS_dist,0.210769,0.210769,0.210769,0.210769,,,Float64
5,FS_sum,-0.0141676,-0.396293,-0.396293,9.43361,,,Float64
6,FS_max,-0.00334192,-0.415792,-0.415792,7.80956,,,Float64
7,FS_max3,-0.0116425,-0.417149,-0.417149,11.6341,,,Float64
8,SS_sum,-0.0269077,-0.292356,-0.292356,7.09575,,,Float64
9,SS_max,-0.0206734,-0.306429,-0.306429,6.38357,,,Float64


In [ ]:
filter(row -> row.SURVERSE == 1, data_set)

In [ ]:
plot(data_set, x=:FS_sum, y=:SS_sum, color=:SURVERSE)

In [21]:
n_to_random_sample = 10000 - size(data_set, 1);
new_entries = similar(data_set, nrow(data_set))

for (i, row) in enumerate(eachrow(data_set))
    new_entries[i, :] = row[:]
end

for i=1:n_to_random_sample
    should_surverse = rand() > 0.45 ? 1 : 0;
    cols = filter(row -> row.SURVERSE == should_surverse, data_set);
    idx_row = convert(Int64, trunc(rand() * size(cols, 1))) + 1;
    
    entry = cols[idx_row, :];
    generate_new_entry(new_entries, entry);
end

In [22]:
describe(new_entries)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1097,,Date
2,SURVERSE,0.5022,0,1.0,1,,,Int64
3,FS_dist,0.145981,0.145981,0.145981,0.145981,,,Float64
4,SS_dist,0.210769,0.210769,0.210769,0.210769,,,Float64
5,FS_sum,0.924133,-0.475435,0.103208,11.2705,,,Float64
6,FS_max,0.812522,-0.498894,0.21335,9.2605,,,Float64
7,FS_max3,0.909081,-0.500545,0.233173,13.8941,,,Float64
8,SS_sum,0.679512,-0.350798,-0.209116,8.4984,,,Float64
9,SS_max,0.573324,-0.36769,-0.161992,7.59169,,,Float64


In [ ]:
filter(row -> row.SURVERSE == 1, new_entries)

In [ ]:
plot(new_entries, x=:FS_sum, y=:SS_sum, color=:SURVERSE)

In [24]:
CSV.write("data/parsed/oversampled/ouvrage_3260.csv", new_entries)

"data/parsed/oversampled/ouvrage_3260.csv"

## Ouvrage 3350

In [25]:
data_set = CSV.read("data/parsed/ouvrage_3350.csv");
size(data_set)

(729, 10)

In [26]:
describe(data_set)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2015-05-01,,2018-10-31,729,,Date
2,SURVERSE,0.203018,0,0.0,1,,,Int64
3,FS_dist,0.0927373,0.0927373,0.0927373,0.0927373,,,Float64
4,SS_dist,0.12027,0.12027,0.12027,0.12027,,,Float64
5,FS_sum,-0.0143977,-0.399971,-0.399971,9.38072,,,Float64
6,FS_max,0.00813102,-0.416911,-0.416911,11.7239,,,Float64
7,FS_max3,-0.00337263,-0.425384,-0.425384,10.573,,,Float64
8,SS_sum,-0.00499908,-0.344782,-0.344782,8.20741,,,Float64
9,SS_max,-0.00230434,-0.361747,-0.361747,6.76672,,,Float64


In [31]:
n_to_random_sample = 10000 - size(data_set, 1);
new_entries = similar(data_set, nrow(data_set))

for (i, row) in enumerate(eachrow(data_set))
    new_entries[i, :] = row[:]
end

for i=1:n_to_random_sample
    should_surverse = rand() > 0.47 ? 1 : 0;
    cols = filter(row -> row.SURVERSE == should_surverse, data_set);
    idx_row = convert(Int64, trunc(rand() * size(cols, 1))) + 1;
    
    entry = cols[idx_row, :];
    generate_new_entry(new_entries, entry);
end

In [32]:
describe(new_entries)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2015-05-01,,2018-10-31,729,,Date
2,SURVERSE,0.4995,0,0.0,1,,,Int64
3,FS_dist,0.0927373,0.0927373,0.0927373,0.0927373,,,Float64
4,SS_dist,0.12027,0.12027,0.12027,0.12027,,,Float64
5,FS_sum,0.447021,-0.479872,-0.221476,11.2546,,,Float64
6,FS_max,0.477792,-0.500178,-0.147966,13.9254,,,Float64
7,FS_max3,0.478369,-0.510409,-0.183571,12.6174,,,Float64
8,SS_sum,0.40016,-0.413725,-0.195807,9.61485,,,Float64
9,SS_max,0.393753,-0.434057,-0.148305,8.00407,,,Float64


In [33]:
CSV.write("data/parsed/oversampled/ouvrage_3350.csv", new_entries)

"data/parsed/oversampled/ouvrage_3350.csv"

## Ouvrage 4240

In [34]:
data_set = CSV.read("data/parsed/ouvrage_4240.csv");
size(data_set)

(1100, 10)

In [35]:
describe(data_set)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1100,,Date
2,SURVERSE,0.0627273,0,0.0,1,,,Int64
3,FS_dist,0.149987,0.149987,0.149987,0.149987,,,Float64
4,SS_dist,0.168295,0.168295,0.168295,0.168295,,,Float64
5,FS_sum,-0.030719,-0.358123,-0.358123,8.69198,,,Float64
6,FS_max,-0.0244404,-0.375363,-0.375363,7.8196,,,Float64
7,FS_max3,-0.0315642,-0.373363,-0.373363,10.0955,,,Float64
8,SS_sum,0.0287264,-0.373593,-0.373593,9.06526,,,Float64
9,SS_max,0.0269202,-0.393477,-0.393477,8.6565,,,Float64


In [38]:
n_to_random_sample = 10000 - size(data_set, 1);
new_entries = similar(data_set, nrow(data_set))

for (i, row) in enumerate(eachrow(data_set))
    new_entries[i, :] = row[:]
end

for i=1:n_to_random_sample
    should_surverse = rand() > 0.45 ? 1 : 0;
    cols = filter(row -> row.SURVERSE == should_surverse, data_set);
    idx_row = convert(Int64, trunc(rand() * size(cols, 1))) + 1;
    
    entry = cols[idx_row, :];
    generate_new_entry(new_entries, entry);
end

In [39]:
describe(new_entries)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1100,,Date
2,SURVERSE,0.4956,0,0.0,1,,,Int64
3,FS_dist,0.149987,0.149987,0.149987,0.149987,,,Float64
4,SS_dist,0.168295,0.168295,0.168295,0.168295,,,Float64
5,FS_sum,0.910909,-0.429708,-0.245224,10.4282,,,Float64
6,FS_max,0.685849,-0.450363,-0.199284,9.36396,,,Float64
7,FS_max3,0.818583,-0.44803,-0.229671,12.0734,,,Float64
8,SS_sum,1.06241,-0.448308,0.0913748,10.8615,,,Float64
9,SS_max,0.78605,-0.4721,0.167074,9.7385,,,Float64


In [40]:
CSV.write("data/parsed/oversampled/ouvrage_4240.csv", new_entries)

"data/parsed/oversampled/ouvrage_4240.csv"

## Ouvrage 4350

In [41]:
data_set = CSV.read("data/parsed/ouvrage_4350.csv");
size(data_set)

(1100, 10)

In [42]:
describe(data_set)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1100,,Date
2,SURVERSE,0.0381818,0,0.0,1,,,Int64
3,FS_dist,0.0248354,0.0248354,0.0248354,0.0248354,,,Float64
4,SS_dist,0.139269,0.139269,0.139269,0.139269,,,Float64
5,FS_sum,-0.00633163,-0.396293,-0.396293,9.43361,,,Float64
6,FS_max,-3.24033e-5,-0.415792,-0.415792,7.80956,,,Float64
7,FS_max3,-0.00776021,-0.417149,-0.417149,11.6341,,,Float64
8,SS_sum,-0.00346406,-0.0493965,-0.0493965,1.1989,,,Float64
9,SS_max,-0.00298854,-0.0517744,-0.0517744,1.07857,,,Float64


In [45]:
n_to_random_sample = 10000 - size(data_set, 1);
new_entries = similar(data_set, nrow(data_set))

for (i, row) in enumerate(eachrow(data_set))
    new_entries[i, :] = row[:]
end

for i=1:n_to_random_sample
    should_surverse = rand() > 0.45 ? 1 : 0;
    cols = filter(row -> row.SURVERSE == should_surverse, data_set);
    idx_row = convert(Int64, trunc(rand() * size(cols, 1))) + 1;
    
    entry = cols[idx_row, :];
    generate_new_entry(new_entries, entry);
end

In [46]:
describe(new_entries)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1100,,Date
2,SURVERSE,0.4952,0,0.0,1,,,Int64
3,FS_dist,0.0248354,0.0248354,0.0248354,0.0248354,,,Float64
4,SS_dist,0.139269,0.139269,0.139269,0.139269,,,Float64
5,FS_sum,1.47254,-0.475484,0.612969,11.3028,,,Float64
6,FS_max,1.36495,-0.498915,0.516277,9.30354,,,Float64
7,FS_max3,1.43037,-0.500555,0.556587,13.9251,,,Float64
8,SS_sum,0.162158,-0.059274,-0.0332895,1.4373,,,Float64
9,SS_max,0.138709,-0.0621233,-0.0271794,1.2941,,,Float64


In [47]:
CSV.write("data/parsed/oversampled/ouvrage_4350.csv", new_entries)

"data/parsed/oversampled/ouvrage_4350.csv"

## Ouvrage 4380

In [48]:
data_set = CSV.read("data/parsed/ouvrage_4380.csv");
size(data_set)

(1103, 10)

In [49]:
describe(data_set)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1103,,Date
2,SURVERSE,0.0761559,0,0.0,1,,,Int64
3,FS_dist,0.0401006,0.0401006,0.0401006,0.0401006,,,Float64
4,SS_dist,0.154948,0.154948,0.154948,0.154948,,,Float64
5,FS_sum,-0.0125561,-0.396293,-0.396293,9.43361,,,Float64
6,FS_max,-0.00278184,-0.415792,-0.415792,7.80956,,,Float64
7,FS_max3,-0.0107162,-0.417149,-0.417149,11.6341,,,Float64
8,SS_sum,-0.0102462,-0.116085,-0.116085,2.8175,,,Float64
9,SS_max,-0.00823173,-0.121674,-0.121674,2.53472,,,Float64


In [52]:
n_to_random_sample = 10000 - size(data_set, 1);
new_entries = similar(data_set, nrow(data_set))

for (i, row) in enumerate(eachrow(data_set))
    new_entries[i, :] = row[:]
end

for i=1:n_to_random_sample
    should_surverse = rand() > 0.45 ? 1 : 0;
    cols = filter(row -> row.SURVERSE == should_surverse, data_set);
    idx_row = convert(Int64, trunc(rand() * size(cols, 1))) + 1;
    
    entry = cols[idx_row, :];
    generate_new_entry(new_entries, entry);
end

In [53]:
describe(new_entries)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,DATE,,2013-05-01,,2018-10-31,1103,,Date
2,SURVERSE,0.4971,0,0.0,1,,,Int64
3,FS_dist,0.0401006,0.0401006,0.0401006,0.0401006,,,Float64
4,SS_dist,0.154948,0.154948,0.154948,0.154948,,,Float64
5,FS_sum,0.919445,-0.475547,0.0147312,11.2242,,,Float64
6,FS_max,0.946782,-0.49895,0.189071,9.33231,,,Float64
7,FS_max3,0.946147,-0.500541,0.0819098,13.868,,,Float64
8,SS_sum,0.290811,-0.139282,-0.0700547,3.3743,,,Float64
9,SS_max,0.284895,-0.145969,-0.0333888,3.02945,,,Float64


In [54]:
CSV.write("data/parsed/oversampled/ouvrage_4380.csv", new_entries)

"data/parsed/oversampled/ouvrage_4380.csv"